# Load Path

In [1]:
import sys
import os
from PIL import Image
import shutil

GOOGLE_DEFAULT_PATH="/content/drive/"
GOOGLE_PROJECT_PATH="My Drive/"
GOOGLE_LESSION_PATH="CNN程式設計概念"
GOOGLE_TMP_NAME="My Drive/course_python_tmp/"

GOOGLE_MY_PATH = GOOGLE_DEFAULT_PATH+GOOGLE_PROJECT_PATH+GOOGLE_LESSION_PATH
MY_PATH_TMP = None

# its win32, maybe there is win64 too?
is_windows = sys.platform.startswith('win')

if is_windows:
    if os.path.exists('./tmp'):
      shutil.rmtree('./tmp')
    pass
    os.makedirs('./tmp')
    MY_PATH_TMP = "./tmp/"
else:
    from google.colab import drive
    from PIL import Image

    # 1. mount google drive
    drive.mount(GOOGLE_DEFAULT_PATH, force_remount=True)
    os.chdir(GOOGLE_MY_PATH) #change to this path
    Curr_Path=os.getcwd()
    print(Curr_Path)

    MY_PATH_TMP = GOOGLE_DEFAULT_PATH+GOOGLE_TMP_NAME

    # 2. create tmp
    if not os.path.exists(MY_PATH_TMP):
        os.makedirs(MY_PATH_TMP)
    pass

pass

Curr_Path=os.getcwd()
print(Curr_Path)

C:\Users\marti\OneDrive\桌面\CNN程式設計概念\Brain_Tumor_detection


套件導入

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import random
import os

###########################
from tqdm import tqdm

# pytorch 函式庫
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import torch.optim as optim

# 借用tensorflow的影像處理函式庫
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical

# 用來調整X/Y的函式庫
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# 用來儲存樣本的函式庫
import pickle # serialization

# 載入黑白AI模型

In [10]:
def Image_to_Array(pic, width, height):
    image = load_img(pic, target_size=(width, height),color_mode='grayscale')
    image_array = img_to_array(image)
    image_preprocess = preprocess_input(image_array) # normalize the value of each element in array
    return  image_preprocess
pass

def load_pictures(picdir, width_size, height_size):

    # init variables to store sample
    x_data = []
    y_labels = []

    for dirPath, dirNames, fileNames in os.walk(picdir):
        for f in fileNames:
            # get y (ans)
            ans = int(dirPath[len(picdir):])

            # get x (picture)
            PicDir = os.path.join(dirPath, f)
            NArray_2D  = Image_to_Array(PicDir,width_size,height_size)

            # store y, x
            y_labels.append(ans)
            x_data.append(NArray_2D)
        pass
    pass
    return y_labels, x_data
pass

def encode_data(i_y, i_x, i_dim):
    # convert the data and labels to NumPy arrays
    x_data = np.array(i_x, dtype="float32")
    y_labels = np.array(i_y)
    #################################################
    # perform one-hot encoding on the labels
    # from 3 to [0,0,0,1,0,0,0,0,0,0]
    #################################################
    y_labels3 = to_categorical(y_labels,i_dim)

    return y_labels3, x_data
pass

# 把眾多照片存成一個檔案
def save_sample(i_y, i_x, i_filename):
    f = open(i_filename, 'wb')
    pickle.dump([i_y,i_x], f)
    f.close()
pass

# 把那一個檔案中讀出來眾多照片
def load_sample(i_filename):
    f = open(i_filename, 'rb')
    r = pickle.load(f)
    y_labels = r[0]
    x_data = r[1]
    return y_labels, x_data
pass

# 在眾多照片中隨機抽樣
def get_sample(i_y, i_x, i_sample_size):
    ##############################
    # get training data randomly

    # init x batch and y batch
    x_batch=[]
    y_batch=[]

    sample = random.sample(range(len(i_y)), i_sample_size)

    x_batch = [i_x[i] for i in sample]
    y_batch = [i_y[i] for i in sample]

    return np.array(y_batch), np.array(x_batch)
pass

# 抽樣完後，把照片轉成tensor格式
def Image_to_Tensor(pic, width_size, height_size, depth_size=1):
    image = None
    if depth_size == 1: # 黑白
        image = load_img(pic, target_size=(width_size, height_size),color_mode='grayscale')
    else : # 彩色
        image = load_img(pic, target_size=(width_size,height_size))
    pass
    image_array = img_to_array(image)
    image_preprocess = preprocess_input(image_array) # normalize the value of each element in array
    # x.reshape(batch, width, height, bw=1) color=4
    x = image_preprocess.reshape(-1, width_size,height_size,depth_size)
    x = np.transpose(x, (0, 3, 1, 2))
    x = torch.from_numpy(x).float()
    return x
pass

class Graph_pytorch(nn.Module):
    def __init__(self):
        super(Graph_pytorch, self).__init__() # customize model
        self.var = OrderedDict()

        #self.op = OrderedDict()

        self.op_sampling = OrderedDict()
        self.op_linear = OrderedDict()

        self.lr = 0.001 # learning rate
    pass
pass

def Graph_pytorch_set_input_size(self, i_input_size):
    self.NETWORK_INPUT_FRAME_WIDTH = i_input_size[0]
    self.NETWORK_INPUT_FRAME_HEIGHT = i_input_size[1]
    self.NETWORK_INPUT_FRAME_DEPTH = i_input_size[2]
pass
Graph_pytorch.set_input_size=Graph_pytorch_set_input_size

def Graph_pytorch_set_output_size(self, i_output_dim):
    self.NETWORK_OUTPUT_DIM=i_output_dim
pass
Graph_pytorch.set_output_size=Graph_pytorch_set_output_size


def Graph_pytorch_init_op_sampling(self):

    self.op_sampling['dropout'] = nn.Dropout2d(p=0.5)
    # conv 1
    _layer_1_input = self.NETWORK_INPUT_FRAME_DEPTH
    _layer_1_output = 32
    self.op_sampling['conv_1'] =  nn.Conv2d(in_channels=_layer_1_input,
                      out_channels=_layer_1_output,
                      kernel_size=5,
                      stride=2,
                      padding=0,
                      bias=True)
    # pool 1
    filter_w=2
    filter_h=2
    stride_x=2
    stride_y=2
    self.op_sampling['pool_1']  = nn.MaxPool2d((filter_w, filter_h),
                       stride=(stride_x, stride_y),
                       padding=0)

    # conv 2
    _layer_3_input = _layer_1_output
    _layer_3_output = 64
    self.op_sampling['conv_2'] =  nn.Conv2d(in_channels=_layer_3_input,
                      out_channels=_layer_3_output,
                      kernel_size=5,
                      stride=2,
                      padding=0)

    # pool 2
    self.op_sampling['pool_2'] = nn.MaxPool2d((filter_w, filter_h),
                      stride=(stride_x, stride_y),
                      padding=0)
pass
Graph_pytorch.init_op_sampling = Graph_pytorch_init_op_sampling


def Graph_pytorch_get_linear_relation_layer_input_size(self):

    i_x = np.zeros((1,  self.NETWORK_INPUT_FRAME_DEPTH,
            self.NETWORK_INPUT_FRAME_WIDTH,
            self.NETWORK_INPUT_FRAME_HEIGHT),dtype=float)
    x = self.to_placeholder(i_x)
    # 產出經過samping的樣本
    for index, layer in self.op_sampling.items():
        x = layer(x)
    pass

    # 找出經過sampling的樣本的大小

    _batch, _width, _height, _depth = list(x.size())

    # 這個經過sampling的樣本，需要拉平成一維矩陣，才能丟進去線性關係式
    # 線性關係式: linear relation / full connected layer / fc layer
    # 下面的步驟是找出一維矩陣的大小
    self.Linear_Relation_INPUT_SIZE = _width * _height * _depth
pass
Graph_pytorch.get_linear_relation_layer_input_size = Graph_pytorch_get_linear_relation_layer_input_size

def Graph_pytorch_init_op_linear(self):
    # 建立多項式 (fc2) Y=AX+b，其中有300個x，Y是1x10的矩陣
    _input_shape = self.Linear_Relation_INPUT_SIZE # 須根據sampling後的大小來手動設定
    _ouput_shape = self.NETWORK_OUTPUT_DIM
    _hidden_layer_shape_1 = 256

    self.op_linear['fc1'] = nn.Linear(_input_shape, _hidden_layer_shape_1)
    # self.op_linear['relu_1'] = nn.ReLU()

    self.op_linear['fc2'] = nn.Linear(_hidden_layer_shape_1, _ouput_shape)
    self.op_linear['softmax'] = nn.Softmax(dim=1)
pass
Graph_pytorch.init_op_linear = Graph_pytorch_init_op_linear


def Graph_pytorch_init_optimizer(self):
    # 指定我們的優化器為 Adam，需要告訴優化器
    # variable是誰(也就是告訴優化器A和b是哪個變數)
    _parameters = []

    # for key, value in self.var.items():
    #   _parameters.append({'params':value})
    #pass

    for key, value in self.op_sampling.items():
        _parameters.append({'params':value.parameters()})
    pass

    for key, value in self.op_linear.items():
        _parameters.append({'params':value.parameters()})
    pass

    self.optimizer = optim.Adam(_parameters,
                  lr=self.lr, weight_decay=0)
pass
Graph_pytorch.init_optimizer = Graph_pytorch_init_optimizer


def Graph_pytorch_forward(self, i_x):

    # (batch, bw color, width, height)
    x = torch.reshape(i_x,(-1,self.NETWORK_INPUT_FRAME_DEPTH
            ,self.NETWORK_INPUT_FRAME_WIDTH
            ,self.NETWORK_INPUT_FRAME_HEIGHT))

    for index, value in self.op_sampling.items():
        x = value(x)
        # nn.ReLU(True)(x)
    pass

    _linear_input_shape = self.Linear_Relation_INPUT_SIZE # 須根據sampling後的大小來手動設定
    x = x.view(-1, _linear_input_shape)

    for index, value in self.op_linear.items():
        x = value(x)
    pass
    return x
pass
Graph_pytorch.forward = Graph_pytorch_forward

def Graph_pytorch_to_placeholder(self, i_x):
    return torch.tensor(i_x, requires_grad=True,dtype=torch.float)
pass
Graph_pytorch.to_placeholder = Graph_pytorch_to_placeholder


def Graph_pytorch_backward(self, i_loss):
    # 反向傳播
    i_loss.backward()
    # 更新模型參數
    self.optimizer.step()
    # 清空梯度
    self.optimizer.zero_grad()
pass
Graph_pytorch.backward = Graph_pytorch_backward


def Graph_pytorch_cal_loss(self, Y_sample, Y_prediction):
    error = Y_sample - Y_prediction
    loss = (error ** 2).mean()
    return loss
pass
Graph_pytorch.cal_loss = Graph_pytorch_cal_loss

將圖片設置為.p，以利之後讀取(若已有要訓練的圖片集檔案可跳過此步)

若已有要訓練的圖片集檔案可跳至此步

In [11]:
y_labels,x_data = load_sample('data/Brain_Tumor_classify.p')
y_labels_glioma_tumor,x_data_glioma_tumor = load_sample('data/glioma_tumor.p')
y_labels_meningioma_tumor,x_data_meningioma_tumor = load_sample('data/meningioma_tumor.p')
y_labels_pituitary_tumor,x_data_pituitary_tumor = load_sample('data/pituitary_tumor.p')

環境設置

In [12]:
############################
# 0. environment setting
# size of picture
width_size=60
height_size=80
depth_size=1 # bw=1, color=3
output_dim=4 # bw tumor img class=4 (0~3), 分類

############################
# 2. data cleaning

# 2.1. data cleaning for x and y
y_c_labels, x_c_data=encode_data(y_labels,x_data,output_dim)
y_c_labels_meningioma_tumor, x_c_data_meningioma_tumor=encode_data(y_labels_meningioma_tumor,x_data_meningioma_tumor,output_dim)
y_c_labels_glioma_tumor, x_c_data_glioma_tumor=encode_data(y_labels_glioma_tumor,x_data_glioma_tumor,output_dim)
y_c_labels_pituitary_tumor, x_c_data_pituitary_tumor=encode_data(y_labels_pituitary_tumor,x_data_pituitary_tumor,output_dim)

# 2.2. partition the data into training and testing sets
# partition the data into training and testing sets
# 90% of the data for training and the remaining 10% for testing
(trainX, testX, trainY, testY) = train_test_split(x_c_data, y_c_labels,
          test_size=0.1, stratify=y_c_labels, random_state=42)

(trainX_meningioma_tumor, testX_meningioma_tumor, trainY_meningioma_tumor, testY_meningioma_tumor) = train_test_split(x_c_data_meningioma_tumor, y_c_labels_meningioma_tumor,test_size=0.1, stratify=y_c_labels_meningioma_tumor, random_state=42)

(trainX_glioma_tumor, testX_glioma_tumor, trainY_glioma_tumor, testY_glioma_tumor) = train_test_split(x_c_data_glioma_tumor, y_c_labels_glioma_tumor,
          test_size=0.1, stratify=y_c_labels_glioma_tumor, random_state=42)

(trainX_pituitary_tumor, testX_pituitary_tumor, trainY_pituitary_tumor, testY_pituitary_tumor) = train_test_split(x_c_data_pituitary_tumor, y_c_labels_pituitary_tumor,
          test_size=0.1, stratify=y_c_labels_pituitary_tumor, random_state=42)

In [13]:
# build a training model
#model_meningioma_tumor
model_meningioma_tumor=Graph_pytorch()
model_meningioma_tumor.set_output_size(output_dim)
model_meningioma_tumor.set_input_size([width_size,height_size,depth_size]) # width, height, depth(bw)
#model_meningioma_tumor.init_var()
model_meningioma_tumor.init_op_sampling()
model_meningioma_tumor.get_linear_relation_layer_input_size()
model_meningioma_tumor.init_op_linear()
model_meningioma_tumor.init_optimizer()

In [14]:
# build a training model
# model_glioma_tumor
model_glioma_tumor=Graph_pytorch()
model_glioma_tumor.set_output_size(output_dim)
model_glioma_tumor.set_input_size([width_size,height_size,depth_size]) # width, height, depth(bw)

#model_glioma_tumor.init_var()
model_glioma_tumor.init_op_sampling()
model_glioma_tumor.get_linear_relation_layer_input_size()
model_glioma_tumor.init_op_linear()
model_glioma_tumor.init_optimizer()

In [15]:
# build a training model
# model_pituitary_tumor
model_pituitary_tumor=Graph_pytorch()
model_pituitary_tumor.set_output_size(output_dim)
model_pituitary_tumor.set_input_size([width_size,height_size,depth_size]) # width, height, depth(bw)

# model_pituitary_tumor.init_var()
model_pituitary_tumor.init_op_sampling()
model_pituitary_tumor.get_linear_relation_layer_input_size()
model_pituitary_tumor.init_op_linear()
model_pituitary_tumor.init_optimizer()

聯邦學習模型訓練
under test(每次訓練前分享一次資訊)

In [16]:
# pychart
chart_epoch = []
chart_x_meningioma_tumor = []
chart_x_glioma_tumor = []
chart_x_pituitary_tumor = []

# 取樣
_sample_y, _sample_x = get_sample(testY, testX, 128)
# testing data transform
x_test = _sample_x.reshape(-1, width_size, height_size, depth_size)
x_test = np.transpose(x_test, (0, 3, 1, 2))
# model_meningioma_tumor
X_test_meningioma_tumor = model_meningioma_tumor.to_placeholder(x_test)
Y_test_meningioma_tumor = model_meningioma_tumor.to_placeholder(_sample_y)
# model_glioma_tumor
X_test_glioma_tumor = model_glioma_tumor.to_placeholder(x_test)
Y_test_glioma_tumor = model_glioma_tumor.to_placeholder(_sample_y)
# model_pituitary_tumor
X_test_pituitary_tumor = model_pituitary_tumor.to_placeholder(x_test)
Y_test_pituitary_tumor = model_pituitary_tumor.to_placeholder(_sample_y)

'''
# 模型欲分享的參數
# sampling
sampling_parameters = ['conv_1','conv_2']
# linear
linear_parameters = ['fc1','fc2']
'''

########################
# federal learning
# 待解決問題
# 1. 合併參數的方式，除了avg之外，還有沒有其他方法，以加快學習速度
# 2. 可否不用合併全部參數，只要合併特定參數即可，以減少頻寬耗費
# 3. (meta learning)合併->自我訓練->合併->自我訓練的週期要多少才能最快化學習速度
print("\n start federal learning....", flush=True)

#交換次數及每次交換中訓練次數設置
FL_EXCHANGE_TIMES=300 # 交換資訊次數
FL_SELF_LEARNING_TIMES=90   # 每輪交換資訊後機構各自訓練次數

model_num = 3   # 參與模型數量
NumberOfOneTraining = 16    # batch size for one training

pbar = tqdm(range(FL_EXCHANGE_TIMES))

for j in pbar:
    # 1. start training
    for i in range(FL_SELF_LEARNING_TIMES):

        # model_meningioma_tumor
        y_train_meningioma_tumor, x_train_meningioma_tumor = get_sample(trainY_meningioma_tumor, trainX_meningioma_tumor, NumberOfOneTraining)
        # 轉換矩陣 from (batch, width, height, depth)
        #      to  (batch, depth, width, height)
        x_train_meningioma_tumor = x_train_meningioma_tumor.reshape(-1, width_size, height_size, depth_size)
        x_train_meningioma_tumor = np.transpose(x_train_meningioma_tumor, (0, 3, 1, 2))
        X_s_meningioma_tumor = model_meningioma_tumor.to_placeholder(x_train_meningioma_tumor)
        Y_s_meningioma_tumor = model_meningioma_tumor.to_placeholder(y_train_meningioma_tumor)
        # get y_pred
        y_prediction_meningioma_tumor = model_meningioma_tumor.forward(X_s_meningioma_tumor)
        # get loss = y - y_pred
        loss_meningioma_tumor = model_meningioma_tumor.cal_loss(Y_s_meningioma_tumor, y_prediction_meningioma_tumor)
        # adjust a,b
        model_meningioma_tumor.backward(loss_meningioma_tumor)

        # model_glioma_tumor
        y_train_glioma_tumor, x_train_glioma_tumor = get_sample(trainY_glioma_tumor, trainX_glioma_tumor, NumberOfOneTraining)
        # 轉換矩陣 from (batch, width, height, depth)
        #      to  (batch, depth, width, height)
        x_train_glioma_tumor = x_train_glioma_tumor.reshape(-1, width_size, height_size, depth_size)
        x_train_glioma_tumor = np.transpose(x_train_glioma_tumor, (0, 3, 1, 2))
        X_s_glioma_tumor = model_glioma_tumor.to_placeholder(x_train_glioma_tumor)
        Y_s_glioma_tumor = model_glioma_tumor.to_placeholder(y_train_glioma_tumor)
        # get y_pred
        y_prediction_glioma_tumor=model_glioma_tumor.forward(X_s_glioma_tumor)
        # get loss = y - y_pred
        loss_glioma_tumor=model_glioma_tumor.cal_loss(Y_s_glioma_tumor, y_prediction_glioma_tumor)
        # adjust a,b
        model_glioma_tumor.backward(loss_glioma_tumor)

        # model_pituitary_tumor
        y_train_pituitary_tumor, x_train_pituitary_tumor = get_sample(trainY_pituitary_tumor, trainX_pituitary_tumor, NumberOfOneTraining)
        # 轉換矩陣 from (batch, width, height, depth)
        #      to  (batch, depth, width, height)
        x_train_pituitary_tumor = x_train_pituitary_tumor.reshape(-1, width_size, height_size, depth_size)
        x_train_pituitary_tumor = np.transpose(x_train_pituitary_tumor, (0, 3, 1, 2))
        X_s_pituitary_tumor = model_pituitary_tumor.to_placeholder(x_train_pituitary_tumor)
        Y_s_pituitary_tumor = model_pituitary_tumor.to_placeholder(y_train_pituitary_tumor)
        # get y_pred
        y_prediction_pituitary_tumor=model_pituitary_tumor.forward(X_s_pituitary_tumor)
        # get loss = y - y_pred
        loss_pituitary_tumor=model_pituitary_tumor.cal_loss(Y_s_pituitary_tumor, y_prediction_pituitary_tumor)
        # adjust a,b
        model_pituitary_tumor.backward(loss_pituitary_tumor)
    pass

    # 2. 經訓練後再次分享各機構參數
    for index in model_meningioma_tumor.op_sampling.keys():
        try:
            avg=(model_meningioma_tumor.op_sampling[index].weight.data + model_glioma_tumor.op_sampling[index].weight.data + model_pituitary_tumor.op_sampling[index].weight.data) / model_num
            model_meningioma_tumor.op_sampling[index].weight.data.copy_(avg)
            model_glioma_tumor.op_sampling[index].weight.data.copy_(avg)
            model_pituitary_tumor.op_sampling[index].weight.data.copy_(avg)
        except:
            continue

    # 已主要模型之參數為主，進行參數分享
    for index in model_meningioma_tumor.op_linear.keys():
        try:
            avg=(model_meningioma_tumor.op_linear[index].weight.data + model_glioma_tumor.op_linear[index].weight.data + model_pituitary_tumor.op_linear[index].weight.data) / model_num
            model_meningioma_tumor.op_linear[index].weight.data.copy_(avg)
            model_glioma_tumor.op_linear[index].weight.data.copy_(avg)
            model_pituitary_tumor.op_linear[index].weight.data.copy_(avg)
        except:
            continue

    # 3. testing
    # model_meningioma_tumor
    # get y_pred
    y_test_prediction_meningioma_tumor = model_meningioma_tumor.forward(X_test_meningioma_tumor)
    y_truth_meningioma_tumor = torch.argmax(Y_test_meningioma_tumor,dim=1).cpu().detach().numpy()
    y_pred_meningioma_tumor = torch.argmax(y_test_prediction_meningioma_tumor,dim=1).cpu().detach().numpy()

    accuracy_meningioma_tumor = accuracy_score(y_truth_meningioma_tumor,y_pred_meningioma_tumor)

    # model_glioma_tumor
    # get y_pred
    y_test_prediction_glioma_tumor = model_glioma_tumor.forward(X_test_glioma_tumor)

    y_truth_glioma_tumor = torch.argmax(Y_test_glioma_tumor,dim=1).cpu().detach().numpy()
    y_pred_glioma_tumor = torch.argmax(y_test_prediction_glioma_tumor,dim=1).cpu().detach().numpy()

    accuracy_glioma_tumor = accuracy_score(y_truth_glioma_tumor,y_pred_glioma_tumor)

    # model_pituitary_tumor
    # get y_pred
    y_test_prediction_pituitary_tumor = model_pituitary_tumor.forward(X_test_pituitary_tumor)

    y_truth_pituitary_tumor = torch.argmax(Y_test_pituitary_tumor,dim=1).cpu().detach().numpy()
    y_pred_pituitary_tumor = torch.argmax(y_test_prediction_pituitary_tumor,dim=1).cpu().detach().numpy()

    accuracy_pituitary_tumor=accuracy_score(y_truth_pituitary_tumor,y_pred_pituitary_tumor)

    pbar.set_description(f'Processing {j}, accuracy_meningioma_tumor: {accuracy_meningioma_tumor}, accuracy_glioma_tumor: {accuracy_glioma_tumor}, accuracy_pituitary_tumor: {accuracy_pituitary_tumor} ')

    # 3.3.4. store the statistic chart
    chart_epoch.append(str(j))
    chart_x_meningioma_tumor.append(accuracy_meningioma_tumor)
    chart_x_glioma_tumor.append(accuracy_glioma_tumor)
    chart_x_pituitary_tumor.append(accuracy_pituitary_tumor)
pass

# save log
df = pd.DataFrame (chart_epoch,columns=['epoch'])
df['accuracy_meningioma_tumor'] = chart_x_meningioma_tumor
df['accuracy_glioma_tumor'] = chart_x_glioma_tumor
df['accuracy_pituitary_tumor'] = chart_x_pituitary_tumor

df.to_csv (MY_PATH_TMP+'fedavgmodel_bs16_train90_exchange300_dropout1_fc2_test3.csv', index = False, header=True)


 start federal learning....


Processing 299, accuracy_meningioma_tumor: 0.6015625, accuracy_glioma_tumor: 0.609375, accuracy_pituitary_tumor: 0.6484375 : 100%|██████████| 300/300 [31:15<00:00,  6.25s/it] 


將訓練好模型載入並存入model以方便之後重開可繼續測試

In [28]:
# 6. save model
torch.save(model_meningioma_tumor, MY_PATH_TMP+'FL_pytorch_model_meningioma_tumor.pkl')
torch.save(model_glioma_tumor, MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
torch.save(model_pituitary_tumor, MY_PATH_TMP+'FL_pytorch_model_pituitary_tumor.pkl')

In [29]:
FL_model_meningioma_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
FL_model_glioma_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_glioma_tumor.pkl')
FL_model_pituitary_tumor = torch.load(MY_PATH_TMP+'FL_pytorch_model_pituitary_tumor.pkl')

In [41]:
# 5. testing a picture
PicDir= "data/val/3/image(1).jpg"
#display(Image.open(PicDir))
NArray_2D  = Image_to_Tensor(PicDir,width_size,height_size)

In [42]:
def pred(prediction):
    if prediction==0:
        return 'normal'
    elif prediction==1:
        return 'meningioma_tumor'
    elif prediction==2:
        return 'glioma_tumor'
    elif prediction==3:
        return 'pituitary_tumor'

In [43]:
#FL_model_meningioma_tumor
prediction=FL_model_meningioma_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only meningioma tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")

#FL_model_glioma_tumor
prediction=FL_model_glioma_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only glioma tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")

#FL_model_pituitary_tumor
prediction=FL_model_pituitary_tumor.forward(NArray_2D)
print(prediction.clone().detach().numpy())
print(f"Predictions for models with only pituitary tumor:{pred(np.argmax(prediction.clone().detach().numpy()))}")


[[1.7473752e-18 9.8523621e-08 1.9556297e-05 9.9998033e-01]]
Predictions for models with only meningioma tumor:pituitary_tumor
[[1.7473752e-18 9.8523621e-08 1.9556297e-05 9.9998033e-01]]
Predictions for models with only glioma tumor:pituitary_tumor
[[2.9009050e-20 2.0560962e-09 4.8667541e-09 1.0000000e+00]]
Predictions for models with only pituitary tumor:pituitary_tumor
